# Replication Notebook for JSS paper
### Estimated total running time: 2 hours and 15 minutes (Apple MacBook Pro M1 32Gb RAM)

# EventTimesSampler

In [ ]:
from time import time
nb_start = time()

In [ ]:
from pydts.data_generation import EventTimesSampler
from pydts.examples_utils.plots import add_panel_text
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
ets = EventTimesSampler(d_times=7, j_event_types=2)

In [ ]:
n_observations = 10000
np.random.seed(0)

In [ ]:
observations_df = pd.DataFrame(columns=['Z1', 'Z2', 'Z3'])
observations_df['Z1'] = np.random.binomial(n=1, p=0.5, size=n_observations)
observations_df.loc[observations_df.loc[observations_df['Z1'] == 0].index, 'Z2'] = \
    np.random.normal(loc=72, scale=12, size=n_observations-observations_df['Z1'].sum())
observations_df.loc[observations_df.loc[observations_df['Z1'] == 1].index, 'Z2'] = \
    np.random.normal(loc=82, scale=12, size=observations_df['Z1'].sum())
observations_df['Z3'] = 1 + np.random.poisson(lam=4, size=n_observations)
observations_df.astype(float).round(2).head()

In [ ]:
print(observations_df.astype(float).round(2).head().to_latex())

In [ ]:
fontsize=17
fig, axes = plt.subplots(1,3, figsize=(16, 4))
ax = axes[0]
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
add_panel_text(ax=ax, text='a')
ax.bar(observations_df['Z1'].value_counts().index, observations_df['Z1'].value_counts().values, width=0.4)
ax.set_xlabel('Z1', fontsize=fontsize)
ax.set_ylabel('Number of observations', fontsize=fontsize)

ax = axes[1]
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
add_panel_text(ax=ax, text='b')
ax.hist(observations_df[observations_df['Z1'] == 0]['Z2'], bins=np.arange(30, 130, step=2), color='tab:orange', 
        label=r'$Z_1=0$', alpha=0.4)
ax.hist(observations_df[observations_df['Z1'] == 1]['Z2'], bins=np.arange(30, 130, step=2), color='tab:green', 
        label=r'$Z_1=1$', alpha=0.4)
ax.legend(fontsize=16)
ax.set_xlabel('Z2', fontsize=fontsize)
ax.set_ylabel('Number of observations', fontsize=fontsize)

ax = axes[2]
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
add_panel_text(ax=ax, text='c')
ax.bar(observations_df['Z3'].value_counts().index, observations_df['Z3'].value_counts().values, width=0.4)
ax.set_xlabel('Z3', fontsize=fontsize)
ax.set_ylabel('Number of observations', fontsize=fontsize)

fig.tight_layout()

In [ ]:
prob_lof_at_t = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
observations_df = ets.sample_independent_lof_censoring(observations_df, prob_lof_at_t)
observations_df.head()

In [ ]:
ets = EventTimesSampler(d_times=7, j_event_types=2)
censoring_coef_dict = {
    "alpha": {
        0: lambda t: -0.3 - 0.3 * np.log(t),
    },
    "beta": {
        0: -np.log([8, 0.95, 6]),
    }
}

observations_df = ets.sample_hazard_lof_censoring(observations_df, censoring_coef_dict)
observations_df.head()

In [ ]:
observations_df['C'].value_counts()

In [ ]:
coefficients_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 1.4, 3]),
        2: -np.log([1, 0.95, 2])
    }
}
observations_df = ets.sample_event_times(observations_df, coefficients_dict)
observations_df.head()

In [ ]:
observations_df['T'].value_counts()

In [ ]:
observations_df['J'].value_counts()

In [ ]:
observations_df = ets.update_event_or_lof(observations_df)
tmp = observations_df.astype({'Z1': int, 'Z2': float, 'Z3': int, 'X': int, 
                        'C': int, 'J': int, 'T': int}).round(2).head()[['Z1', 'Z2', 'Z3', 'T', 'C', 'X', 'J']]
tmp

In [ ]:
print(tmp.to_latex())

# Simulation Study - Data Preparation

For simplicity of presentation, we considered $M=2$ competing events, though PyDTS can handle any number of competing events as long as there are enough observed failures of each failure type, at each discrete time point.

Here, $d=30$ discrete time points, $n=50,000$ observations, and $Z$ with 5 covariates. Failure times of observations were generated based on the model:

$$
\lambda_{j}(t|Z) = \frac{\exp(\alpha_{jt}+Z^{T}\beta_{j})}{1+\exp(\alpha_{jt}+Z^{T}\beta_{j})}
$$

with 

$\alpha_{1t} = -1 -0.3 \log(t)$, 

$\alpha_{2t} = -1.75 -0.15\log(t)$, $t=1,\ldots,d$,

$\beta_1 = (-\log 0.8, \log 3, \log 3, \log 2.5, \log 2)$, 

$\beta_{2} = (-\log 1, \log 3, \log 4, \log 3, \log 2)$. 

Censoring time for each observation was sampled from a discrete uniform distribution, i.e. $C_i \sim \mbox{Uniform}\{1,...,d+1\}$.

Our goal is estimating $\{\alpha_{11},\ldots,\alpha_{1d},\beta_1^T,\alpha_{21},\ldots,\alpha_{2d},\beta_2^T\}$ (70 parameters in total) along with the standard error of the estimators.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
import warnings
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])
    }
}

n_patients = 50000
n_cov = 5

In [ ]:
patients_df = generate_quick_start_df(n_patients=n_patients, n_cov=n_cov, d_times=30, j_events=2, 
                                      pid_col='pid', seed=0, censoring_prob=0.8, 
                                      real_coef_dict=real_coef_dict)

patients_df.head()

## Checking the Data

Both estimation methods require enough observed failures of each failure type, at each discrete time point. Therefore, the first step is to make sure this is in fact the case with the data at hand.

As shown below, in our example, the data comply with this requirement. 

Preprocessing suggestions for cases when the data do not comply with this requirement are shown in Data Regrouping Example.

In [ ]:
from pydts.examples_utils.plots import plot_events_occurrence
plot_events_occurrence(patients_df)

In [ ]:
patients_df.groupby(['J', 'X'])['pid'].count().unstack('J')

# Estimating with DataExpansionFitter

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
import warnings
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])
    }
}

n_patients = 50000
n_cov = 5

In [ ]:
patients_df = generate_quick_start_df(n_patients=n_patients, n_cov=n_cov, d_times=30, j_events=2, 
                                      pid_col='pid', seed=0, censoring_prob=0.8, 
                                      real_coef_dict=real_coef_dict)
patients_df.head()

## Estimation

In the following we apply the estimation method of Lee et al. (2018). Note that the data dataframe must not contain a column named 'C'.

In [ ]:
from pydts.fitters import DataExpansionFitter
fitter = DataExpansionFitter()
fitter.fit(df=patients_df.drop(['C', 'T'], axis=1))

fitter.print_summary()

In [ ]:
from pydts.examples_utils.plots import plot_first_model_coefs
plot_first_model_coefs(models=fitter.event_models, times=fitter.times, train_df=patients_df, n_cov=5)

In [ ]:
summary = fitter.event_models[1].summary()
summary_df = pd.DataFrame([x.split(',') for x in summary.tables[1].as_csv().split('\n')])
summary_df.columns = summary_df.iloc[0]
summary_df = summary_df.iloc[1:].set_index(summary_df.columns[0])
lee_beta1_summary = summary_df.iloc[-5:]
lee_beta1_summary

In [ ]:
summary = fitter.event_models[2].summary()
summary_df = pd.DataFrame([x.split(',') for x in summary.tables[1].as_csv().split('\n')])
summary_df.columns = summary_df.iloc[0]
summary_df = summary_df.iloc[1:].set_index(summary_df.columns[0])
lee_beta2_summary = summary_df.iloc[-5:]
lee_beta2_summary

Full prediction is given by the method predict_cumulative_incident_function()

The input is a pandas.DataFrame() containing for each observation the covariates columns which were used in the fit() method (Z1-Z5 in our example).

The following columns will be added:

1. The overall survival at each time point t
2. The hazard for each failure type $j$ at each time point t
3. The probability of event type $j$ at time t
4. The Cumulative Incident Function (CIF) of event type $j$ at time t

In the following, we provide predictions for the individuals with ID values (pid) 0, 1 and 2. We transposed the output for easy view.

In [ ]:
pred_df = fitter.predict_cumulative_incident_function(
    patients_df.drop(['J', 'T', 'C', 'X'], axis=1).head(3)).set_index('pid').T
pred_df.index.name = ''
pred_df.columns = ['ID=0', 'ID=1', 'ID=2']

In [ ]:
from pydts.examples_utils.plots import plot_example_pred_output
plot_example_pred_output(pred_df)

In [ ]:
print(pred_df)

# Estimating with TwoStagesFitter

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
import warnings
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])
    }
}

n_patients = 50000
n_cov = 5

In [ ]:
patients_df = generate_quick_start_df(n_patients=n_patients, n_cov=n_cov, d_times=30, j_events=2, 
                                      pid_col='pid', seed=0, censoring_prob=0.8, 
                                      real_coef_dict=real_coef_dict)
patients_df.head()

## Estimation

In the following we apply the estimation method of Meir et al. (2022). Note that the data dataframe must not contain a column named 'C'.

In [ ]:
from pydts.fitters import TwoStagesFitter
new_fitter = TwoStagesFitter()
new_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1))

new_fitter.print_summary()

In [ ]:
from pydts.examples_utils.plots import plot_second_model_coefs
plot_second_model_coefs(new_fitter.alpha_df, new_fitter.beta_models, new_fitter.times, n_cov=5)

## Standard Error of the Regression Coefficients

In [ ]:
print(new_fitter.get_beta_SE())

In [ ]:
new_fitter.plot_all_events_alpha()

In [ ]:
new_fitter.plot_all_events_beta()

In [ ]:
twostep_beta_summary = new_fitter.get_beta_SE()
twostep_beta1_summary = twostep_beta_summary.iloc[:,[0,1]]
twostep_beta2_summary = twostep_beta_summary.iloc[:,[2,3]]
twostep_beta_summary

## Standard Error of the Regression Coefficients Comparison Table

In [ ]:
lee_beta1_summary.round(3)

In [ ]:
lee_beta1_summary = lee_beta1_summary.iloc[:, [0,1]].round(3)
lee_beta2_summary = lee_beta2_summary.iloc[:, [0,1]].round(3)
lee_beta1_summary.columns = pd.MultiIndex.from_tuples([('Lee et al.', 'Estimate'), ('Lee et al.', 'SE')])
lee_beta2_summary.columns = pd.MultiIndex.from_tuples([('Lee et al.', 'Estimate'), ('Lee et al.', 'SE')])
beta_summary_comparison = pd.concat([lee_beta1_summary, lee_beta2_summary], axis=0)
beta_summary_comparison.index = [r'$\beta_{11}$', r'$\beta_{12}$', r'$\beta_{13}$', r'$\beta_{14}$', r'$\beta_{15}$',
                                 r'$\beta_{21}$', r'$\beta_{22}$', r'$\beta_{23}$', r'$\beta_{24}$', r'$\beta_{25}$']
twostep_beta1_summary.columns = pd.MultiIndex.from_tuples([('two-step', 'Estimate'), ('two-step', 'SE')])
twostep_beta2_summary.columns = pd.MultiIndex.from_tuples([('two-step', 'Estimate'), ('two-step', 'SE')])
tmp = pd.concat([twostep_beta1_summary.round(3), twostep_beta2_summary.round(3)], axis=0)
tmp.index = [r'$\beta_{11}$', r'$\beta_{12}$', r'$\beta_{13}$', r'$\beta_{14}$', r'$\beta_{15}$',
             r'$\beta_{21}$', r'$\beta_{22}$', r'$\beta_{23}$', r'$\beta_{24}$', r'$\beta_{25}$']
beta_summary_comparison = pd.concat([beta_summary_comparison, tmp], axis=1)
beta_summary_comparison.index.name =  r'$\beta_{jk}$'

true_col = -np.log([0.8, 3, 3, 2.5, 2, 1, 3, 4, 3, 2])   
beta_summary_comparison.insert(loc=0, column='True', value=true_col)
beta_summary_comparison.astype(float).round(3)

In [ ]:
print(beta_summary_comparison.astype(float).round(3).to_latex(escape=False))

## Regularization

It is possible to add regularization when estimating the Beta coefficients. It is done by using the CoxPHFitter (Lifelines) penalizer and l1_ratio arguments, which can be passed using the fit_beta_kwargs argument to the fit() method. The added regularization term is of the form:
$$
\mbox{Penalizer} \cdot \Bigg( \frac{1-\mbox{L1_ratio}}{2}||\beta||_{2}^{2} + \mbox{L1_ratio} ||\beta||_1 \Bigg)
$$
Examples for adding L1, L2 and Elastic Net regularization are followed.

### L1

In [ ]:
L1_regularized_fitter = TwoStagesFitter()

fit_beta_kwargs = {
    'model_kwargs': {
        1: {'penalizer': 0.003, 'l1_ratio': 1},
        2: {'penalizer': 0.005, 'l1_ratio': 1}
}}

L1_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

L1_regularized_fitter.get_beta_SE()

### L2

In [ ]:
L2_regularized_fitter = TwoStagesFitter()

fit_beta_kwargs = {
    'model_kwargs': {
        1: {'penalizer': 0.003, 'l1_ratio': 0},
        2: {'penalizer': 0.005, 'l1_ratio': 0}
}}

L2_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

L2_regularized_fitter.get_beta_SE()

### Elastic Net

In [ ]:
EN_regularized_fitter = TwoStagesFitter()

fit_beta_kwargs = {
    'model_kwargs': {
        1: {'penalizer': 0.003, 'l1_ratio': 0.5},
        2: {'penalizer': 0.005, 'l1_ratio': 0.5}
}}

EN_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

EN_regularized_fitter.get_beta_SE()

### Separated Penalty Coefficients

The above methods can be applied with a separate penalty coefficient to each of the covariates by passing a vector (with same length as the number of covariates) to the penalizer keyword instead of a scalar. For example, applying L2 regularization only to covariates Z1, Z2 can be done as follows:

In [ ]:
L2_regularized_fitter = TwoStagesFitter()

fit_beta_kwargs = {
    'model_kwargs': {
        1: {'penalizer': np.array([0.01, 0.01, 0.01, 0.01, 0]), 'l1_ratio': 0},
        2: {'penalizer': np.array([0.05, 0.05, 0.05, 0.05, 0]), 'l1_ratio': 0}
}}

L2_regularized_fitter.fit(df=patients_df.drop(['C', 'T'], axis=1), fit_beta_kwargs=fit_beta_kwargs)

L2_regularized_fitter.get_beta_SE()

## Prediction

Full prediction is given by the method predict_cumulative_incident_function()

The input is a pandas.DataFrame() containing for each observation the covariates columns which were used in the fit() method (Z1-Z5 in our example).

The following columns will be added:

1. The overall survival at each time point t
2. The hazard for each failure type $j$ at each time point t
3. The probability of event type $j$ at time t
4. The Cumulative Incident Function (CIF) of event type $j$ at time t

In the following, we provide predictions for the individuals with ID values (pid) 0, 1 and 2. We transposed the output for easy view.

In [ ]:
pred_df = new_fitter.predict_cumulative_incident_function(
    patients_df.drop(['J', 'T', 'C', 'X'], axis=1).head(3)).set_index('pid').T
pred_df.index.name = ''
pred_df.columns = ['ID=0', 'ID=1', 'ID=2']

In [ ]:
from pydts.examples_utils.plots import plot_example_pred_output
plot_example_pred_output(pred_df)

In [ ]:
pred_df

# Comparing the Estimation Methods

## Introduction

We conducted a simulation study demonstrating the performances of Meir et al. (2022) [1] and comparing it with that of Lee et al. (2018) [2]. 

The data was generated in the same way as in Usage Example section, i.e. $M=2$ competing events, $n=50,000$ observations, Z with 5 covariates and right censoring. 

Failure times were generated based on 

$$
\lambda_{j}(t|Z) = \frac{\exp(\alpha_{jt}+Z^{T}\beta_{j})}{1+\exp(\alpha_{jt}+Z^{T}\beta_{j})}
$$

with 

$\alpha_{1t} = -1 -0.3 \log(t)$, 

$\alpha_{2t} = -1.75 -0.15\log(t)$, $t=1,\ldots,d$,

$\beta_1 = (-\log 0.8, \log 3, \log 3, \log 2.5, \log 2)$, 

$\beta_{2} = (-\log 1, \log 3, \log 4, \log 3, \log 2)$. 

Censoring time for each observation was sampled from a discrete uniform distribution, i.e. $C_i \sim \mbox{Uniform}\{1,...,d+1\}$. 

We repeated this procedure for $d \in (15, 30, 45, 60, 100)$ and report the results in Meir et al. (2022) [1]. For each value of $d$, the results are based on 100 replications. 

We showed that both estimation methods perform very well in terms of bias and provide highly similar results in terms of point estimators and their standard errors. However, the computational running time of our approach is 1.5-3.5 times shorter depending on $d$, where the improvement factor increases as a function of $d$.

## Estimation Replications

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
import warnings
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])
    }
}

n_patients = 50000
n_cov = 5


# In JSS paper the results of 100 replications are presented.
# We present the results here for 5 replication for reasonable running time.
# For higher number of d_times, longer running time is expected.
# Estimated running time for each replication when d=30: 45 seconds

replications = 5  
d_times = 30 

In [ ]:
from pydts.fitters import repetitive_fitters
rep_dict, times_dict, counts_df = repetitive_fitters(rep=replications, n_patients=n_patients, n_cov=n_cov, 
                                                     d_times=30, j_events=2, pid_col='pid', test_size=0.25, 
                                                     verbose=0, real_coef_dict=real_coef_dict, censoring_prob=0.8,
                                                     allow_fails=2)

## Comparing Standard Error of Lee et al. (2018) and Meir et al. (2022)

In [ ]:
from pydts.examples_utils.plots import plot_reps_coef_std

new_res_dict = plot_reps_coef_std(rep_dict, True)

## Comparison of the Estimated Coefficients

In [ ]:
from pydts.examples_utils.plots import plot_models_coefficients
a = new_res_dict['alpha']
b = new_res_dict['beta']
times = [t+1 for t in list(a[1].reset_index().index)]
n_cov = 5
plot_models_coefficients(a, b, times, counts_df)

## Computational Time Comparison

In [ ]:
from pydts.examples_utils.plots import plot_times

plot_times(times_dict)

In [ ]:
beta_comparison_table = pd.DataFrame(index=['True', 'Mean (Lee et al.)', 'SE (Lee et al.)',
                                              'Mean (two-step)', 'SE (two-step)'])
for j in [1, 2]:
    for i in range(1, 6):
        tmp_df = pd.DataFrame()
        for idx, k in enumerate(sorted(rep_dict.keys())):
            lee = rep_dict[k]['beta'][j].loc[f"Z{i}_{j}"]['Lee']
            ours = rep_dict[k]['beta'][j].loc[f"Z{i}_{j}"]['Ours']
            true = rep_dict[k]['beta'][j].loc[f"Z{i}_{j}"]['real']
            row = pd.Series({'True': true, 'Lee': lee, 'Ours': ours}, name=f"Z{i}_{j}_{k}")
            tmp_df = pd.concat([tmp_df, row], axis=1)
        beta_row = pd.Series({
            'True': tmp_df.iloc[0,0],
            'Mean (Lee et al.)': tmp_df.iloc[1].mean(), 
            'SE (Lee et al.)': tmp_df.iloc[1].std(),
            'Mean (two-step)': tmp_df.iloc[2].mean(),
            'SE (two-step)': tmp_df.iloc[2].std()
        }, name=f'Z{i}_{j}')
        beta_comparison_table = pd.concat([beta_comparison_table, beta_row], axis=1)


In [ ]:
beta_comparison_table = beta_comparison_table.round(3).T
beta_comparison_table.columns = pd.MultiIndex.from_tuples(
    [('True', ''), ('Lee et al.', 'Estimate'), ('Lee et al.', 'SE'), ('two-step', 'Estimate'), ('two-step', 'SE')])
beta_comparison_table.index = [r'$\beta_{11}$', r'$\beta_{12}$', r'$\beta_{13}$', r'$\beta_{14}$', r'$\beta_{15}$',
                               r'$\beta_{21}$', r'$\beta_{22}$', r'$\beta_{23}$', r'$\beta_{24}$', r'$\beta_{25}$']
beta_comparison_table.index.name = r'$\beta_{jk}$'
beta_comparison_table

In [ ]:
print(beta_comparison_table.to_latex(escape=False))

In [ ]:
from pydts.examples_utils.plots import plot_reps_coef_std

new_res_dict = plot_reps_coef_std(rep_dict)

# Penalty Grid Search

In [ ]:
from sklearn.model_selection import train_test_split
from pydts.model_selection import PenaltyGridSearch
from pydts.cross_validation import PenaltyGridSearchCV
train_df, test_df = train_test_split(patients_df, train_size=0.8, random_state=1)

In [ ]:
penalizers = np.exp([-2, -3, -4, -5, -6])
grid_search = PenaltyGridSearch()
optimal_set = grid_search.evaluate(train_df, test_df, l1_ratio=1, 
                                   penalizers=penalizers,
                                   metrics=['IBS', 'GBS', 'IAUC', 'GAUC']) 

In [ ]:
print(np.log(optimal_set))

In [ ]:
print(np.log(grid_search.convert_results_dict_to_df(grid_search.global_bs).idxmin().values[0]))

In [ ]:
penalizers = np.exp([-2, -3, -4, -5, -6])
grid_search_cv = PenaltyGridSearchCV()
results_df = grid_search_cv.cross_validate(patients_df, l1_ratio=1, 
                                penalizers=penalizers, n_splits=5, 
                                metrics=['IBS', 'GBS', 'IAUC', 'GAUC'])

In [ ]:
results_df['Mean'].idxmax()

In [ ]:
results_df_ = results_df.copy().reset_index()
results_df_.iloc[:, :2] = np.log(results_df_.iloc[:, :2])
results_df_ = results_df_.set_index(['level_0','level_1'])
results_df_.index.set_names(['log(eta_1)', 'log(eta_2)'], inplace=True)
print(results_df_)

In [ ]:
nb_end = time()
print(int(nb_end-nb_start))

## References

[1] Meir, Tomer\*, Gutman, Rom\*, and Gorfine, Malka, "PyDTS: A Python Package for Discrete-Time Survival Analysis with Competing Risks" (2022)

[2] Lee, Minjung and Feuer, Eric J. and Fine, Jason P., "On the analysis of discrete time competing risks data", Biometrics (2018) doi: 10.1111/biom.12881